## INITIALIZE


In [88]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
from model_set import Custom1DCNN
import capilab_dataset2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

label_keys = {'ball':1,'box':2,'pen':3,'no':0}
fname = ['Datasets/Lai_JulyData.mat', 'Datasets/Takahashi_JulyData.mat', 'Datasets/Sugiyama_JulyData.mat']
Xx, yy = capilab_dataset2.get(fname) 
print(Xx.shape)
try:
    X, x_test, y, y_test = train_test_split(Xx, yy, test_size = .1, stratify = yy, random_state = 420)
    print(X.shape, y.shape, x_test.shape, y_test.shape)
except Exception as e:
    print(e)


(1800, 1000, 19)
(1620, 1000, 19) (1620, 4) (180, 1000, 19) (180, 4)


In [4]:
#STFU
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [60]:
t = 12.2

print('{:.3f}'.format(t))

12.200


In [89]:
from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
kfold = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 420)

f1 = []
recall = []
prec = []
acc = []
verbose =False

target = [1,3, 5]
x_train = X[:,:,target]
x_test_1 = x_test[:,:,target]
for k, (train, val) in enumerate(kfold.split(X, np.argmax(y, axis = 1))):
    print('Training on Fold {} ({}/{})'.format(k + 1, train.shape, val.shape))
    
    classifier_optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
    classifier_loss = tf.keras.losses.CategoricalCrossentropy()
    clf = Custom1DCNN(inp_shape=(X.shape[1], X.shape[2]), output_classes=y.shape[1])
    clf.compile(optimizer = classifier_optimizer, loss= classifier_loss , metrics=['accuracy'])
    es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0.001, patience=9, restore_best_weights=True, verbose = False) 
    
    clf.fit(x_train[train], y[train],
            batch_size=32, 
            epochs = 70, 
            verbose = verbose, 
            validation_data = (x_train[val],y[val]),
            callbacks = [es])
    y_preds = clf.predict(x_test_1, verbose = verbose)
    predicted = np.argmax(y_preds, axis=1)
    ground_truth = np.argmax(y_test, axis=1)
        
    acc.append(accuracy_score(ground_truth, predicted))
    f1.append(f1_score(ground_truth, predicted, average = 'macro'))
    prec.append(precision_score(ground_truth, predicted, average = 'macro'))
    recall.append(recall_score(ground_truth, predicted, average = 'macro'))
    
    print('Fold {} : ACC {:.4f} | RECALL {:.4f} | PRECISION {:.4f} | F1 {:.4f}'.format(
        k + 1, acc[-1], recall[-1], prec[-1], f1[-1]
    ))
    



Training on Fold 1 ((1458,)/(162,))
Fold 1 : ACC 0.3722 | RECALL 0.3722 | PRECISION 0.4053 | F1 0.3697
Training on Fold 2 ((1458,)/(162,))
Fold 2 : ACC 0.4111 | RECALL 0.4111 | PRECISION 0.3185 | F1 0.3556
Training on Fold 3 ((1458,)/(162,))
Fold 3 : ACC 0.4611 | RECALL 0.4611 | PRECISION 0.4783 | F1 0.4643
Training on Fold 4 ((1458,)/(162,))


/home/capiadmin/anaconda3/envs/dqn_chsel/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1327: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Fold 4 : ACC 0.2611 | RECALL 0.2611 | PRECISION 0.1577 | F1 0.1297
Training on Fold 5 ((1458,)/(162,))
Fold 5 : ACC 0.4056 | RECALL 0.4056 | PRECISION 0.3815 | F1 0.3849
Training on Fold 6 ((1458,)/(162,))
Fold 6 : ACC 0.4056 | RECALL 0.4056 | PRECISION 0.4117 | F1 0.3969
Training on Fold 7 ((1458,)/(162,))
Fold 7 : ACC 0.4000 | RECALL 0.4000 | PRECISION 0.4120 | F1 0.3922
Training on Fold 8 ((1458,)/(162,))
Fold 8 : ACC 0.4167 | RECALL 0.4167 | PRECISION 0.4280 | F1 0.4087
Training on Fold 9 ((1458,)/(162,))
Fold 9 : ACC 0.3944 | RECALL 0.3944 | PRECISION 0.3868 | F1 0.3830
Training on Fold 10 ((1458,)/(162,))
Fold 10 : ACC 0.4111 | RECALL 0.4111 | PRECISION 0.3720 | F1 0.3813


In [93]:
tf.keras.backend.clear_session()